In [1]:
#https://www.madisonsquareclothing.com/collections/shop-all

In [97]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
html=requests.get('https://www.madisonsquareclothing.com/collections/shop-all').content
soup=BeautifulSoup(html,'html.parser')

In [3]:
#max page
max_page=int(soup.select('div.pagination a')[2].text)
print("max_page_pagination:",max_page)

max_page_pagination: 20


In [9]:
#concat the url 
urls=[]
url='https://www.madisonsquareclothing.com/collections/shop-all?page='
for i in range(1, max_page+1):
    urls.append(url+str(i))

In [28]:
host_url='https://www.madisonsquareclothing.com'
model_urls=[]

#get all models url
for url in urls:
    html=requests.get(url).content
    soup=BeautifulSoup(html,'html.parser')
    for a in soup.find_all('a', class_='grid-product__image-link', href=True):
        model_urls.append(host_url+a['href'])
print("done")

done


In [29]:
model_urls

['https://www.madisonsquareclothing.com/collections/shop-all/products/bells-out-poncho-grey',
 'https://www.madisonsquareclothing.com/collections/shop-all/products/kamilla-cardigan-charcoal',
 'https://www.madisonsquareclothing.com/collections/shop-all/products/mona-cardigan-khaki',
 'https://www.madisonsquareclothing.com/collections/shop-all/products/molly-knit-khaki',
 'https://www.madisonsquareclothing.com/collections/shop-all/products/molly-knit-camel',
 'https://www.madisonsquareclothing.com/collections/shop-all/products/mckenzie-skivvy-coffee',
 'https://www.madisonsquareclothing.com/collections/shop-all/products/mckenzie-skivvy-grey',
 'https://www.madisonsquareclothing.com/collections/shop-all/products/callista-knit-grey',
 'https://www.madisonsquareclothing.com/collections/shop-all/products/virgo-knit-khaki',
 'https://www.madisonsquareclothing.com/collections/shop-all/products/mona-cardigan-black',
 'https://www.madisonsquareclothing.com/collections/shop-all/products/mckenzie

In [102]:
#create dataframe columns
col_names =  ['product_name', 'price', 'metadata','product_id','image_download']
df=pd.DataFrame(columns=col_names)

,product_name,price,metadata,product_id,image_download


In [103]:
for url in model_urls:
    print("url:",url)
    #you also can add header infomation in requests.get()
    html=requests.get(url).content
    soup=BeautifulSoup(html,'html.parser')
    product_name=soup.select('div.product-single__meta>h1.product-single__title')[0].text.strip()
    price=soup.select('span.money')[0].text.strip().replace('$','')
    #some product has no metadata
    try:
        metadata=soup.find_all('p',attrs={'style':'text-align: center;'})[0].text.strip()
        if len(metadata)==0:
            soup.find_all('h3',attrs={'style':'text-align: center;'})[0].text.strip()
    except:
        metadata=""
    #save model images
    idx=0
    product_id=""
    for img in soup.find_all('img',attrs={'class':'product-single__photo'}):
        response = requests.get('http:'+img['src'], stream=True)
        with open(img['src'].split('=')[-1]+'_'+str(idx)+'.jpg', 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response
        idx+=1
        product_id=img['src'].split('=')[-1]
    #append to dataframe
    df=df.append({'product_name': product_name, 'price': price, \
                  'metadata': metadata,'product_id':product_id,\
                  'image_download':'OK'}, ignore_index=True)
print("done")

done


In [104]:
df.to_csv('static_webpage.csv', header=True, index=False)